## Importing the libraries


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf

## Loading the dataset

In [6]:
# Load your time series data into a DataFrame
# Replace 'your_data.csv' with the actual file or data source.
df = pd.read_csv('../Datasets/train.csv')
df = df.iloc[:1826,3:]


## Augmented Dickey Fuller test for finding out whether the dataset is stationary or not

In [7]:
# Perform the ADF test
result = adfuller(df['sales'])

# Extract and print the test statistics and p-value
adf_statistic = result[0]
p_value = result[1]
print(f'ADF Statistic: {adf_statistic}')
print(f'p-value: {p_value}')

# Compare the p-value to a significance level (e.g., 0.05) for the inference
if p_value <= 0.05:
    print("Reject the null hypothesis: Data is stationary.")
else:
    print("Fail to reject the null hypothesis: Data is non-stationary.")


ADF Statistic: -3.157670556332807
p-value: 0.022569380626571333
Reject the null hypothesis: Data is stationary.


## The Autocorrelation plot tells that there is a correlation between the lags since it has a positive correlation